In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split,  KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

from statsmodels.genmod.generalized_linear_model import GLM
import statsmodels.api as sm
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

sns.set()
pd.set_option('display.precision', 3)

In [ ]:
# First, be sure to download the dataset
filename: str = 'dataset.csv'
dataset = pd.read_pickle('dataRead_processed.pkl.bz2', compression='bz2')
dataset.shape

## Linear regression

In [ ]:
dataset.drop(columns=['total_passengers_2015',
                      'total_passengers_and_non_passengers_2022',
                      'total_passengers_and_non_passengers_2015',
                      'total_passengers_and_non_passengers_2022_std',
                      'total_passengers_2022_bx',
                      'total_passengers_and_non_passengers_2022_bx',
                      'total_passengers_2022_BC',
                      'total_passengers_2022_min_max',
                      'total_passengers_and_non_passengers_2022_min_max',
                      'total_passengers_2022_std',
                      ], inplace=True)
dataset.dtypes

In [ ]:
# Split the dataset into training and testing sets

X = dataset.loc[:,dataset.columns != 'total_passengers_2022']
y = dataset['total_passengers_2022']

X_train, X_test, y_train, y_test = train_test_split(X, # Features
                                                    y, # Target
                                                    test_size=0.33, # Percentage of the dataset to be used as test set
                                                    random_state=10 # Seed
                                                    )

In [ ]:
def minimum_preprocessing(X, y):
    print('Original shape:{}'.format(X.shape))
    categorical_columns = X.dtypes[X.dtypes == 'category'].index.values
    object_columns = X.dtypes[X.dtypes == 'object'].index.values
    # We kill categorical columns
    X=X.drop(columns=categorical_columns)
    X=X.drop(columns=object_columns)
    print('Droped category: {}'.format(categorical_columns))
    print('Droped object: {}'.format(object_columns))
    # We remove missing values
    X=X.dropna()
    y=y[X.index]
    print('New shape:{}'.format(X.shape))
    return X, y

In [ ]:
X_train, y_train = minimum_preprocessing(X_train,y_train)
X_test, y_test = minimum_preprocessing(X_test,y_test)

In [ ]:
dataset.columns

In [ ]:
# We instantiate a linear regression. 
lr = LinearRegression() # From sklearn

# Fit the model using the training set
lr.fit(X_train,y_train)

# Predict the target using the training set
y_pred = lr.predict(X_train)

weights = lr.coef_
intercept = lr.intercept_
# You can access to some info about the model, like the weights.
print('Coefficients: \n', weights[:10])
print('Intercept: \n', intercept)

In [ ]:
# You can also use sklearn implementation
mean_square_error_sk = mean_squared_error(y_train, y_pred)
mean_square_error_sk

# MSE is the average of the square of the errors. The larger the number, the larger the error.
norm_mse_sk = 1-r2_score(y_train, y_pred)

R_squared_sk = r2_score(y_train,y_pred) 

mean_square_error_sk, norm_mse_sk, R_squared_sk

### Cross-validation

In [ ]:
cross_val_metrics = pd.DataFrame(columns=['MSE', 'norm_MSE', 'R2'])


kf = KFold(n_splits=5)
i=1
for train_index, test_index in kf.split(X_train):
    print('Split {}: \n\tTest Folds: [{}] \n\tTrain Folds {}'.format(i, i, [j for j in range(1,6) if j != i]))
    
    x_train_fold = X_train.values[train_index]
    y_train_fold = y_train.values[train_index]
    x_test_fold = X_train.values[test_index,:]
    y_test_fold = y_train.values[test_index]
    
    lr = LinearRegression()
    lr.fit(x_train_fold,y_train_fold)
    y_pred_fold = lr.predict(x_test_fold)
    fold_mse =mean_squared_error(y_test_fold, y_pred_fold)
    fold_nmse =  1-r2_score(y_test_fold, y_pred_fold)
    fold_r2 = r2_score(y_test_fold, y_pred_fold)
    print('\tMSE: {} NMSE: {} R2: {}'.format(fold_mse,fold_nmse, fold_r2) )

    cross_val_metrics.loc['Fold {}'.format(i), :] = [fold_mse,fold_nmse, fold_r2]
    i+=1
    
    
cross_val_metrics.loc['Mean',:] = cross_val_metrics.mean()
cross_val_metrics

In [ ]:
lr = LinearRegression()
lr.fit(X_train,y_train)
folds_r2 = cross_val_score(lr, X_train,y_train, cv=5, scoring='r2')
lr_r2 = np.mean(folds_r2) 
folds_r2, lr_r2

### Ridge regression

In [ ]:
cross_val_metrics = pd.DataFrame(columns=['MSE', 'norm_MSE', 'R2'])

def get_best_ridge_regression_parameter(hyperparameters: list[float],
                                        X_train: pd.DataFrame,
                                        y_train: pd.Series,
                                        cv=5
                                        ) -> float:
    """
    Cross validation for Ridge regression.
    Parameters:
    hyperparameters: list of lambda values to be tested
    X_train: Training set
    y_train: Target
    cv: Number of folds
    
    return best model hyperparameter
    """
    ridge_cross_val_metrics = pd.DataFrame(columns=['mean MSE', 'mean norm_MSE', 'mean R2'])


    for lambda_val in hyperparameters:
        kf = KFold(n_splits=cv)
        i=1
        cv_mse = []
        cv_nmse = []
        cv_r2 = []
        
        for train_index, test_index in kf.split(X_train):
            print('Hyperparameter: {}\n\tSplit {}: \n\t\tTest Folds: [{}] \n\t\tTrain Folds {}'.format(lambda_val, i, i, [j for j in range(1,6) if j != i]))
            
            x_train_fold = X_train.values[train_index]
            y_train_fold = y_train.values[train_index]
            x_test_fold = X_train.values[test_index,:]
            y_test_fold = y_train.values[test_index]

            lr = Ridge(alpha=lambda_val)
            lr.fit(x_train_fold,y_train_fold)
            y_pred_fold = lr.predict(x_test_fold)
            
            fold_mse = mean_squared_error(y_test_fold, y_pred_fold)
            fold_nmse = 1-r2_score(y_test_fold, y_pred_fold)
            fold_r2 = r2_score(y_test_fold, y_pred_fold)
            cv_mse.append(fold_mse)
            cv_nmse.append(fold_nmse)
            cv_r2.append(fold_r2)
            print('\t\tMSE: {} NMSE: {} R2: {}'.format(fold_mse,fold_nmse, fold_r2) )
        
        ridge_cross_val_metrics.loc['Lambda={}'.format(lambda_val),:] = [np.mean(cv_mse),np.mean(cv_nmse),np.mean(cv_r2)]

    ridge_cross_val_metrics.sort_values(by='mean R2',ascending=False)
    
    # Return the best model
    best_lambda = ridge_cross_val_metrics.idxmax(axis=0)['mean R2']
    
    # Extract number from string
    best_lambda = float(best_lambda.split('=')[1])
    
    return best_lambda

def cross_validation_ridge_regression(best_hyperparameter: float,
                                      X_train: pd.DataFrame,
                                      y_train: pd.Series,
                                      cv=5
                                      ) -> Ridge:
    
    kf = KFold(n_splits=5)
    i=1
    
    for train_index, test_index in kf.split(X_train):        
        x_train_fold = X_train.values[train_index]
        y_train_fold = y_train.values[train_index]
        x_test_fold = X_train.values[test_index,:]
        y_test_fold = y_train.values[test_index]
        
        lr = LinearRegression()
        lr.fit(x_train_fold,y_train_fold)
        y_pred_fold = lr.predict(x_test_fold)
        fold_mse =mean_squared_error(y_test_fold, y_pred_fold)
        fold_nmse =  1-r2_score(y_test_fold, y_pred_fold)
        fold_r2 = r2_score(y_test_fold, y_pred_fold)

        cross_val_metrics.loc['Fold {}'.format(i), :] = [fold_mse,fold_nmse, fold_r2]
        i+=1
        
        
    cross_val_metrics.loc['Mean',:] = cross_val_metrics.mean()
    cross_val_metrics
    return 
    
# lambdas = [1e-10,1e-5,1e-4,1e-3,1e-2,0.1, 0.5,1,5,10,50,100]
lambdas = np.logspace(start = -4, stop = 1.1, num = 100, base = 10.0)


get_best_ridge_regression_parameter(hyperparameters=lambdas, X_train=X_train, y_train=y_train)


In [ ]:
# Train model with the best hyperparameter
best_lambda = get_best_ridge_regression_parameter(hyperparameters=lambdas, X_train=X_train, y_train=y_train)

print(f"Best lambda: {best_lambda}")

# Use cross validation
ridge = Ridge(alpha=best_lambda)
ridge.fit(X_train,y_train, )
y_pred = ridge.predict(X_train)


## Manual implementation

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(X_train)   # computes means and stdevs for each column in X_train
X_train_scaled = scaler.transform(X_train)             # substracts mean and divides by stdev (estimated from training)
X_test_scaled = scaler.transform(X_test)               # substracts mean and divides by stdev (estimated from training)

X_train_scaled[:,0] = 1   # undo transformation for all-1 column
X_test_scaled[:,0] = 1   # undo transformation for all-1 column

print(X_train_scaled.mean(axis=0))
print(X_test_scaled.mean(axis=0))

In [ ]:
from sklearn.metrics import mean_squared_error

lambdas = np.logspace(start = -4, stop = 1.1, num = 100, base = 10.0)
results = []

X = X_train_scaled.copy()
y = y_train.copy()
n = y.shape[0]
d = X.shape[1]


for l in lambdas:
    XtX = X.T @ X
    XtX_inv = np.linalg.inv( XtX + l * np.identity(n=d))
    coefs = (XtX_inv) @ X.T @ y
    hatmat = X @ XtX_inv @ X.T
    trace_hatmat = np.trace(hatmat)
    y_pred = X @ coefs
    
    loocv = 1/n * np.sum([((y.iloc[i] - y_pred[i]) / (1 - hatmat[i,i]))**2 for i in range(n)])
    
    mse = mean_squared_error(y, y_pred)
    gcv = mse / (1 - trace_hatmat/n)**2
    results.append([l, mse, loocv, gcv])

df = pd.DataFrame(results, columns = ['lambda', 'training_mse', 'loocv', 'gcv']) 
df.sort_values(by='loocv')

In [ ]:
df.plot(kind='line', x='lambda')

In [ ]:
best_lambda = df.loc[df['loocv'].idxmin()]['lambda']

print(f'best lambda value: {best_lambda:.4f}')

# apply formula with "best lambda"
theta_vector = np.linalg.inv( X_train_scaled.T @ X_train_scaled + best_lambda * np.identity(n=d)) @ X_train_scaled.T @ y

In [ ]:
from sklearn.linear_model import RidgeCV

print(f'there are {X_train_scaled.shape[0]} training examples.')
results = []
for k in range(2, 6+1):
    ridge = RidgeCV(alphas=lambdas, fit_intercept=False, cv=k)   #k-fold cross-val
    clf = ridge.fit(X_train_scaled, y_train)
    results.append([k, clf.alpha_])

## "efficient"  way:
ridge = RidgeCV(alphas=lambdas, fit_intercept=False, cv=None)
clf = ridge.fit(X_train_scaled, y_train)
results.append(['efficient', clf.alpha_])

pd.DataFrame(results, columns=['cross-val method (k)', 'best lambda'])